In [ ]:
import requests
import json

import pandas as pd
from time import sleep
from tqdm.notebook import tqdm

In [ ]:
def pretty_json(json_object):
    print(json.dumps(json_object, indent=2))

In [ ]:
test_df = pd.read_csv("../data/raw/task1_test_no_label.csv")
train_df = pd.read_csv("../data/raw/task1_train_dataset.csv")

In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
train_df.head()

In [ ]:
api_key = ""
with open ("../data/api_key.txt", "r") as apikey_file:
    api_key=apikey_file.readlines()[0].strip()
api_endpoint = "https://api.core.ac.uk/v3/"

In [ ]:
def get_entity(url_fragment):
    headers={"Authorization":"Bearer "+api_key}
    response = requests.get(api_endpoint + url_fragment, headers=headers)
    if response.status_code == 200:
        return response.json(), response.elapsed.total_seconds()
    elif response.status_code == 410:
        return {}, None
    elif response.status_code == 429:
        sleep(300)
        response = requests.get(api_endpoint + url_fragment, headers=headers)
        if response.status_code == 200:
            return response.json(), response.elapsed.total_seconds()
        else:
            return {}, None
    else:
        print(f"Error code {response.status_code}, {response.content}")
        return {}, None

In [ ]:
outputs = [get_entity(f"outputs/{x}")[0] for x in train_df['core_id'].tolist()[:20]]

In [ ]:
pd.DataFrame.from_dict(outputs)

In [ ]:
outputs = []
for index_i, x in tqdm(enumerate(train_df['core_id'].tolist()),total=(len(train_df))):
    if (index_i+1) % 45 == 0:
        sleep(300)
    result = get_entity(f"outputs/{x}")[0]
    outputs.append(result)

In [ ]:
df = pd.DataFrame.from_dict(outputs)

In [ ]:
df.to_csv("../../data/processed/train_core-sample.csv")

# Load data from CORE

In [ ]:
df = pd.read_csv("../data/processed/train_core-11200.csv")

In [ ]:
print("% of papers which are available (True)")
(~df.id.isnull()).value_counts() / len(df)

In [ ]:
print("% of papers for which there is full text (True)")
(~df.fullText.isnull()).value_counts() / len(df)

In [ ]:
print("% of papers which have pdfs available (True)")
(~df.sourceFulltextUrls.isnull()).value_counts() / len(df)

In [ ]:
print("% of papers which have references available (True)")
(~df.references.isnull()).value_counts() / len(df)